In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pylab import savefig
import mdtraj as md
import MDAnalysis as mda
import copy
from __future__ import print_function
#import nglview as nv

%matplotlib inline

In [23]:
work_dir = '/mnt/storage1/adam/CGRP/steuer/trans_S512_S516_Ac_CGRP_8-18_NH2/'

In [24]:
prmtop_name = 'trans_S512_S516.prmtop'

In [25]:
nc_file = "heat2.nc"

In [26]:
#nc_file2 = "prod2.nc"

In [27]:
#nc_file3 = "prod3.nc"

In [28]:
#nc_file4 = "prod4.nc"

In [29]:
#nc_file5 = "prod5.nc"

In [30]:
system_name = 'S512-S516 $\it{trans}$ Ac-CGRP(8-18)-NH$_2$'

In [31]:
step = ''
#spellings (because dyslexic): heating, equilibration, production

### Loading the full trajectory - saving last frame

In [32]:
traj1 = md.load_netcdf(work_dir+nc_file, top=work_dir+prmtop_name, stride = None)

In [33]:
#traj2 = md.load_netcdf(work_dir+nc_file2, top=work_dir+prmtop_name, stride = 10)

In [34]:
#traj3 = md.load_netcdf(work_dir+nc_file3, top=work_dir+prmtop_name, stride = 10)

In [35]:
#traj4 = md.load_netcdf(work_dir+nc_file4, top=work_dir+prmtop_name, stride = 10)

In [36]:
#traj5 = md.load_netcdf(work_dir+nc_file5, top=work_dir+prmtop_name, stride = 10)

In [37]:
traj = traj1

In [38]:
traj 

<mdtraj.Trajectory with 54 frames, 24081 atoms, 5987 residues, and unitcells at 0x706de384bf70>

Loading a trajectory with stride

In [ ]:
#traj_chunk = md.load_netcdf(work_dir+nc_file, top=work_dir+prmtop_name, stride=10)

In [ ]:
#traj_chunk

##### Saving the last frame of the trajectory as a PDB file (only for production step)

In [39]:
traj[-1].save_pdb(work_dir+'S512-S516_trans_Ac-CGRP(8-18)-NH2_after_heat2_last-frame.pdb')

##### Saving .dcd trajectories for PyContact input

In [ ]:
#traj.save_dcd(work_dir+'prod.dcd')

In [ ]:
#traj[::10].save_dcd(work_dir+'prod_every10thframe.dcd') # saving the trajectory in dcd format every 10 frames

#### Finding the protein residues/indices

In [ ]:
res_list = []
for res in traj.topology.residues:
    res_list.append(res)

In [ ]:
res_list

In [ ]:
res_list_strings = []

for res in res_list:
    res_list_strings.append(str(res))

In [ ]:
protein_res = [res for res in traj.topology.residues if res.is_protein or res.name == 'NHE' or res.name == 'ACE' or res.name == 'AKS' or res.name == 'AKR' 
or res.name == 'TDD'  or res.name == 'UES' or res.name == 'UKS' or res.name == 'HRA' or res.name == 'HSA' or res.name == 'ORA' or res.name == 'ORG' 
or res.name == 'R51' or res.name == 'R52' or res.name == 'S51' or res.name == 'S52' or res.name == 'O2O' or res.name == 'AZG']

In [ ]:
protein_res_strings = []

for i in protein_res:
    protein_res_strings.append(str(i))

print(protein_res_strings)

Useful for viewing specific amino acids:

In [ ]:
for i in protein_res_strings:
    if 'NHE' in i:
        print(i)

### Loading the trajectory for the peptide only

##### Getting peptide atom indices

##### Change number of residues in the peptide, e.g. 31 for a peptide with both ACE and NH2 caps, 30 for a peptide with only NH2 cap, 32 for a peptide with ACE, NH2 caps and TRP

In [ ]:
peptide_res_number = 3

#### For peptides containing NHE cap

In [ ]:
for res in res_list_strings:
    if res[0:3] == 'NHE':
        print(res)
        c_terminus = int(res[3:7])

In [ ]:
peptide_first_res = c_terminus - peptide_res_number
print('the first residue of the peptide is', res_list[peptide_first_res])
print('the last residue of the peptide is', res_list[c_terminus])

In [ ]:
n_ter = int(str(res_list[peptide_first_res])[3:7])
c_ter = int(str(res_list[c_terminus])[3:7])
print(n_ter, c_ter)

In [ ]:
# mdtraj selection for the peptide
peptide_selection_mdtraj = 'resid '+str(n_ter)+' to '+str(c_ter)
print('mdtraj selection for the peptide: '+peptide_selection_mdtraj)

In [ ]:
# MDAnalysis selection for the peptide (for use in PyContact)
# MDAnalysis numbers residues/atoms from 1, whereas mdtraj numbers residues/atoms from 0

peptide_selection_mdanalysis = 'resid '+str(n_ter+1)+':'+str(c_ter+1)
print('MDAnalysis selection for the peptide: '+peptide_selection_mdanalysis)

In [ ]:
peptide_atom_idx = traj.topology.select(peptide_selection_mdtraj) #change this depending on the first and last residues

#atoms and residues in mdtraj - and python in general - are listed from 0 value 
#the PDB file will be +1 in value

In [ ]:
peptide_res = res_list_strings[n_ter:c_ter+1]
peptide_res

#### For CGRP peptides without NHE cap

In [ ]:
#for res in res_list_strings:
#    if res[0:3] == 'VAL':
#        print(res)
#        if res == 'VAL456':
#           n_ter = int(res[3:6])
#            print('the first residue of the peptide is', res)

In [ ]:
#peptide_last_res = n_ter + peptide_res_number
#c_ter = int(str(res_list[peptide_last_res])[3:6])
#print('the first residue of 8-18 CGRP is', res_list[n_ter])
#print('the last residue of 8-18 CGRP is', res_list[c_ter])

In [ ]:
#peptide_selection_mdtraj = 'resid '+str(n_ter)+' to '+str(c_ter)
#print('mdtraj selection for the peptide: '+peptide_selection_mdtraj)

In [ ]:
# MDAnalysis selection for the peptide (for use in PyContact)
# MDAnalysis numbers residues/atoms from 1, whereas mdtraj numbers residues/atoms from 0

#peptide_selection_mdanalysis = 'resid '+str(n_ter+1)+':'+str(c_ter+1)
#print('MDAnalysis selection for the peptide: '+peptide_selection_mdanalysis)

In [ ]:
#peptide_atom_idx = traj.topology.select(peptide_selection_mdtraj) #this will be changed depending on the first and last residues

# atoms and residues in mdtraj - and python in general - are listed from 0 value 
# MDAnalysis and the PDB file will be +1 in value

In [ ]:
#peptide_res = res_list_strings[n_ter:c_ter+1]
#peptide_res

#### Finding the receptor residues/indices:

In [ ]:
receptor_res = copy.deepcopy(protein_res_strings)

for i in peptide_res:
    if i in receptor_res:
        receptor_res.remove(i)

print(receptor_res)

Finding the NME caps in the receptor (if any)

In [ ]:
nme_res = [res for res in traj.topology.residues if res.name == 'NME']
nme_res

In [ ]:
nme_res_list = []

for nme in nme_res:
    nme=str(nme)
    nme_res_list.append(int(nme[3:6]))

nme_res_list

In [ ]:
# Check that the receptor and peptide have the correct number of residues

print(len(protein_res_strings))
print(len(peptide_res))
print(len(receptor_res))

Saving the receptor atom indices:

In [ ]:
# finding out if the peptide is sandwiched between the receptor residues or is after the receptor at the end of the PDB file

if n_ter == int(receptor_res[-1][3:6])+1:
    print('the peptide is after the receptor, at the end of the PDB file')
    receptor_first_res = int(receptor_res[0][3:6])
    receptor_last_res = int(receptor_res[-1][3:6])
    receptor_selection_mdtraj = 'resid '+str(receptor_first_res)+' to '+str(receptor_last_res)
    receptor_selection_mdanalysis = 'resid '+str(receptor_first_res+1)+':'+str(receptor_last_res+1)
    print('select the receptor with:', receptor_selection_mdtraj)
    print('for PyContact, select the receptor with: ', receptor_selection_mdanalysis)
else:
    print('the peptide is sandwiched between the receptor residues in the PDB file')
    receptor_chunk1_first_res = int(receptor_res[0][3:6])
    receptor_chunk1_last_res = n_ter - 1
    receptor_chunk2_first_res = c_ter + 1
    receptor_chunk2_last_res = int(receptor_res[-1][3:6])
    receptor_selection_mdtraj = 'resid ' +str(receptor_chunk1_first_res)+ ' to ' + str(receptor_chunk1_last_res) +' or resid '+str(receptor_chunk2_first_res)+' to '+str(receptor_chunk2_last_res)
    receptor_selection_mdanalysis = 'resid '+str(receptor_chunk1_first_res+1)+':'+str(receptor_chunk1_last_res+1)+' or resid '+str(receptor_chunk2_first_res+1)+':'+str(receptor_chunk2_last_res+1)
    print('select the receptor with:', receptor_selection_mdtraj)
    print('for PyContact, select the receptor with: ', receptor_selection_mdanalysis)


In [ ]:
receptor_atom_idx = traj.topology.select(receptor_selection_mdtraj)

In [ ]:
# receptor_atom_idx = traj.topology.select('resid 0 to 114 or resid 146 to 486') 
# change this depending on the receptor_res list, e.g. resid 0 to 455 if the peptide is at the end of the PDB file
# or resid 0 to 114 and resid 146 to 486 if the peptide is in between
# always check with the receptor residues printed above

In [ ]:
print(receptor_atom_idx)

#### After the correct peptide and receptor have been selected (check everything looks right), it's time for renumbering and further analysis

Renumbering the residues because the CGRP peptide starts from Val8

In [ ]:
peptide_res_renumbered = []

if peptide_res[0][0:3] == 'ACE' and peptide_res[1][0:3] == 'VAL':
    idx=7
    ace_cap = 1
elif peptide_res[0][0:3] == 'ACE' and peptide_res[1][0:3] == 'TRP':
    idx=6
    ace_cap = 1
elif peptide_res[0][0:3] == 'ACE' and peptide_res[1][0:3] == 'HIS':
    idx=6
    ace_cap = 1
elif peptide_res[0][0:3] == 'ACE' and peptide_res[1][0:3] == 'PHE':
    idx=6
    ace_cap = 1
elif peptide_res[0][0:3] == 'ACE' and peptide_res[1][0:3] == 'TYR':
    idx=6
    ace_cap = 1
elif peptide_res[0][0:3] == 'ACE' and peptide_res[1][0:3] == 'ALA':
    idx=6
    ace_cap = 1
elif peptide_res[0][0:3] == 'ACE' and peptide_res[1][0:3] == 'ORG':
    idx=7
    ace_cap = 1
elif peptide_res[0][0:3] == 'ACE' and peptide_res[1][0:3] == 'ORA':
    idx=7
    ace_cap = 1
elif peptide_res[0][0:3] == 'ACE' and peptide_res[1][0:3] == 'GLU':
    idx=0
    ace_cap = 1
elif peptide_res[0][0:3] == 'VAL':
    idx=8
    ace_cap = 0
elif peptide_res[0][0:3] == 'GLN':
    idx=6
    ace_cap = 0
elif peptide_res[0][0:3] == 'PHE':
    idx=6
    ace_cap = 0
elif peptide_res[0][0:3] == 'ARG':
    idx=1
    ace_cap = 0
elif peptide_res[0][0:3] == 'ALA':
    idx=1
    ace_cap = 0
elif peptide_res[0][0:3] == 'BZU':
    idx=27
    ace_cap = 0
elif peptide_res[0][0:3] == 'HIS':
    idx=1
    ace_cap = 0
elif peptide_res[0][0:3] == 'CYS':
    idx=1
    ace_cap = 0
elif peptide_res[0][0:3] == 'GLU':
    idx=1
    ace_cap = 0
elif peptide_res[0][0:3] == 'TYR':
    idx=1
    ace_cap = 0
elif peptide_res[0][0:3] == 'GLY':
    idx=1
    ace_cap = 0

#for 8-37 peptide, change to 6 if Trp before Val, change to 8 if there is no Ac cap, change to 7 if there is an Ac cap
#for 27-37 peptide, change to 27 if there is no Ac cap, change to 26 if there is an Ac cap
#for 1-7 peptide, change to 1
for i in peptide_res:
    peptide_res_renumbered.append(i[0:3]+' '+str(idx))
    idx +=1

peptide_res_renumbered

In [ ]:
peptide_length=len(peptide_res)
#peptide_length_no_caps = peptide_length - ace_cap #last residue is NH2 cap. change to 1 if Ac cap is present
#print('the length of the peptide is', peptide_length_no_caps) 

In [ ]:
#if peptide_res_number != peptide_length_no_caps:
#    print('check that the peptide selection is correct')

#### Loading peptide trajectory

In [ ]:
traj_peptide1 = md.load_netcdf(work_dir+nc_file, top=work_dir+prmtop_name, atom_indices=peptide_atom_idx, stride = None)

In [ ]:
#traj_peptide2 = md.load_netcdf(work_dir+nc_file2, top=work_dir+prmtop_name, atom_indices=peptide_atom_idx, stride = 10)

In [ ]:
#traj_peptide3 = md.load_netcdf(work_dir+nc_file3, top=work_dir+prmtop_name, atom_indices=peptide_atom_idx, stride = 10)

In [ ]:
#traj_peptide4 = md.load_netcdf(work_dir+nc_file4, top=work_dir+prmtop_name, atom_indices=peptide_atom_idx, stride = 10)

In [ ]:
traj_peptide = traj_peptide1

In [ ]:
traj_peptide

#### Calculating secondary structure and helicity for the peptide

In [ ]:
sec_struct = md.compute_dssp(traj_peptide, simplified = True)
print(sec_struct)

In [ ]:
frames = len(sec_struct)
print('number of frames is', frames)

In [ ]:
sec_struct[-1] #get secondary structure of specific frame

Saving the secondary structure of each residue in the struct_list:

In [ ]:
idx = 0
struct_list=[]
resid_list=[]


for idx in range(0, peptide_length):
    for frame in sec_struct:
        resid_list.append(frame[idx])
    struct_list.append(resid_list)
    resid_list=[]

In [ ]:
count = 0
helicity_per_residue = []
helicity = 0

for res in struct_list:
    if 'NA' in res:
        print('excluding the ACE and NH2 caps')   
    else:
        for snapshot in res:
            if snapshot == 'H':
                count+=1
            helicity = (count/frames)*100
        helicity_per_residue.append(helicity)
        count=0

Calculating the helicity per residue - excluding caps:

In [ ]:
helicity_per_residue 

Creating the %helicity per residue plot:

In [ ]:
fig,ax = plt.subplots()
ax.plot(peptide_res_renumbered[ace_cap:peptide_res_number], helicity_per_residue) 
# [1:peptide_length_no_caps] if ACE present
# [:peptide_length_no_caps] if no ACE 
# ax.plot(peptide_res_renumbered[ace_cap:peptide_res_number+1] for maxadilan

#ax.set(ylabel='%helicity per residue', title='%average helicity per residue - '+step)

plt.xticks(rotation=85, fontsize=5)

ax.set_ylim([0.0, 105.0])
plt.gcf().set_size_inches(15,5)

fig.savefig(work_dir+'helicity_per_residue_'+step+'.png', bbox_inches='tight') 
plt.show()

Block analysis for helicity

In [ ]:
# block analysis for helicity - include error in plots
# first need to check how errors change as a function of block size (for each residue?)

# tests for Gly21
# need to substitute all elements in the struct list -> 100 if H, 0 otherwise


#res_helicity = []

#for i in struct_list[13]:
#    if i == 'H':
#        res_helicity.append(100)
#    else:
#        res_helicity.append(0)

#res_helicity

In [ ]:
#n_blocks = 10
#bsize = int(len(res_helicity)/n_blocks) # 10 blocks of 50 frames each
#bav = np.zeros(n_blocks)

#for i in range(n_blocks) : 
#    bav[i] = sum( res_helicity[i*bsize:(i+1)*bsize] ) / bsize 
#mean = sum( bav ) / n_blocks
#var = (float(n_blocks)/float(n_blocks-1))*( sum(bav*bav) / n_blocks - mean*mean )
#print( "From block averaging of size", bsize, "average is", mean, "and error is", np.sqrt( var/n_blocks) )

In [ ]:
#avg_value = []
#block_number = []
#err = []

#for i in range(10,frames+10,10):
#    n_blocks=i
#    bsize = int(len(res_helicity)/n_blocks)
#    bav = np.zeros(n_blocks)
#    for i in range(n_blocks) : 
#        bav[i] = sum( res_helicity[i*bsize:(i+1)*bsize] ) / bsize 
#        mean = sum( bav ) / n_blocks
#        var = (float(n_blocks)/float(n_blocks-1))*( sum(bav*bav) / n_blocks - mean*mean )
#    avg_value.append(mean)
#    block_number.append(n_blocks)
#    err.append(np.sqrt( var/n_blocks))
#    print( "From block averaging of blocks", n_blocks, "size", bsize, "average is", mean, "and error is", np.sqrt( var/n_blocks) )


In [ ]:
#fig,ax = plt.subplots()
#ax.set(xlabel='number of blocks', ylabel='average % helicity', title='Gly 21')

#ax.set_ylim([0.0, 100.0])
#ax.errorbar(block_number, avg_value, yerr=err, fmt='none', ecolor='papayawhip', elinewidth=0.6, capsize=1)
#ax.plot(block_number, avg_value, color='darkorange') 

#plt.show()


In [ ]:
peptide_helicity=[]
res_helicity = []

for res in range(len(peptide_res_renumbered)):
    for i in struct_list[res]:
        if i == 'H':
            res_helicity.append(100)
        else:
            res_helicity.append(0)
    peptide_helicity.append(res_helicity)
    res_helicity=[]

In [ ]:
# plotting the average and error as a function of block size

#for res in range(len(peptide_res_renumbered)):
#    res_helicity=peptide_helicity[res]
#    avg_value = []
#    block_number = []
#    err = []
#    for i in range(10,frames+10,50):
#        n_blocks=i
#        bsize = int(len(res_helicity)/n_blocks)
#        bav = np.zeros(n_blocks)
#        for i in range(n_blocks) : 
#            bav[i] = sum( res_helicity[i*bsize:(i+1)*bsize] ) / bsize 
#            mean = sum( bav ) / n_blocks
#            var = (float(n_blocks)/float(n_blocks-1))*( sum(bav*bav) / n_blocks - mean*mean )
#        avg_value.append(mean)
#        block_number.append(n_blocks)
#        err.append(np.sqrt( var/n_blocks))

#    fig,ax = plt.subplots()
#    ax.set(xlabel='number of blocks', ylabel='average % helicity', title=peptide_res_renumbered[res])

#    #ax.set_ylim([0.0, 105.0])
#    ax.errorbar(block_number, avg_value, yerr=err, fmt='none', ecolor='black', elinewidth=0.6, capsize=1)
#    ax.plot(block_number, avg_value, color='darkorange') 

#    plt.show()


In [ ]:
if nc_file == 'heat1.nc':
     n_blocks = 5
else:
     n_blocks = 50

avg_value = []
err = []

for res in range(len(peptide_res_renumbered)):
     res_helicity=peptide_helicity[res]
     bsize = int(len(res_helicity)/n_blocks)
     bav = np.zeros(n_blocks)
     for i in range(n_blocks) : 
          bav[i] = sum( res_helicity[i*bsize:(i+1)*bsize] ) / bsize 
          mean = sum( bav ) / n_blocks
          var = (float(n_blocks)/float(n_blocks-1))*( sum(bav*bav) / n_blocks - mean*mean )
     avg_value.append(mean)
     #block_number.append(n_blocks)
     err.append(np.sqrt( var/n_blocks))


In [ ]:
fig,ax = plt.subplots()
ax.set(ylabel='%helicity per residue', title='%average helicity per residue - ' + system_name)
plt.xticks(rotation=85, fontsize=14)

ax.set_ylim([0.0, 105.0])
plt.gcf().set_size_inches(15,5)

ax.errorbar(peptide_res_renumbered[ace_cap:peptide_res_number], avg_value[ace_cap:peptide_res_number], yerr=err[ace_cap:peptide_res_number], fmt='none', ecolor='black', elinewidth=0.6, capsize=1)
ax.plot(peptide_res_renumbered[ace_cap:peptide_res_number], avg_value[ace_cap:peptide_res_number], color='darkorange') 

fig.savefig(work_dir+'helicity_per_residue_with_errors_'+step+'.png', bbox_inches='tight')
plt.show()

In [ ]:
# checking how correlated the data are

#d2 = res_helicity - mean
#c = np.correlate(d2, d2, 'full')
#c = c[int(c.size/2):]
#c /= (c.var() * np.arange(d2.size, 0, -1))
#plt.xlabel('sample interval')
#plt.ylabel('correlation')
#plt.plot(c[:500])

##### Average helicity from 3 replicates

Loading the peptide trajectories from replicates 2 and 3

In [ ]:
traj_peptide_rep2 = md.load_netcdf(work_dir+'Replicate-2/'+nc_file, top=work_dir+prmtop_name, atom_indices=peptide_atom_idx)
traj_peptide_rep2

In [ ]:
sec_struct_rep2 = md.compute_dssp(traj_peptide_rep2, simplified = True)
print(sec_struct_rep2)

In [ ]:
idx = 0
struct_list=[]
resid_list=[]


for idx in range(0, peptide_length):
    for frame in sec_struct_rep2:
        resid_list.append(frame[idx])
    struct_list.append(resid_list)
    resid_list=[]

In [ ]:
count = 0
helicity_per_residue_rep2 = []
helicity = 0

for res in struct_list:
    if 'NA' in res:
        print('excluding the ACE and NH2 caps')   
    else:
        for snapshot in res:
            if snapshot == 'H':
                count+=1
            helicity = (count/frames)*100
        helicity_per_residue_rep2.append(helicity)
        count=0

In [ ]:
helicity_per_residue_rep2

In [ ]:
traj_peptide_rep3 = md.load_netcdf(work_dir+'Replicate-3/'+nc_file, top=work_dir+prmtop_name, atom_indices=peptide_atom_idx)
traj_peptide_rep3

In [ ]:
sec_struct_rep3 = md.compute_dssp(traj_peptide_rep3, simplified = True)
print(sec_struct_rep3)

In [ ]:
idx = 0
struct_list=[]
resid_list=[]


for idx in range(0, peptide_length):
    for frame in sec_struct_rep3:
        resid_list.append(frame[idx])
    struct_list.append(resid_list)
    resid_list=[]

In [ ]:
count = 0
helicity_per_residue_rep3 = []
helicity = 0

for res in struct_list:
    if 'NA' in res:
        print('excluding the ACE and NH2 caps')   
    else:
        for snapshot in res:
            if snapshot == 'H':
                count+=1
            helicity = (count/frames)*100
        helicity_per_residue_rep3.append(helicity)
        count=0

In [ ]:
helicity_per_residue_rep3

Plotting the helicities of the three replicates together

In [ ]:
fig,ax = plt.subplots()
ax.plot(peptide_res_renumbered[ace_cap:peptide_res_number], helicity_per_residue, color='royalblue', label='Replicate 1')
ax.plot(peptide_res_renumbered[ace_cap:peptide_res_number], helicity_per_residue_rep2, color='tomato', label='Replicate 2')
ax.plot(peptide_res_renumbered[ace_cap:peptide_res_number], helicity_per_residue_rep3, color='seagreen', label='Replicate 3')
# [1:peptide_length_no_caps] if ACE present
# [:peptide_length_no_caps] if no ACE 

ax.set(ylabel='%helicity per residue', title='%average helicity per residue - ' + system_name)
ax.legend(bbox_to_anchor=(1,1), loc='upper left')

plt.xticks(rotation=85, fontsize=14)

ax.set_ylim([0.0, 105.0])
plt.gcf().set_size_inches(15,5)

fig.savefig(work_dir+'helicity_per_residue_allreps_'+step+'.png', bbox_inches='tight') 
plt.show()

Calculating st.dev. from the 3 replicates and plotting average helicity with error bars

In [ ]:
helicity_tuple = []

for i in range(len(helicity_per_residue)):
    helicity_tuple.append((helicity_per_residue[i], helicity_per_residue_rep2[i], helicity_per_residue_rep3[i]))

avg_helicity_per_residue = []
avg_helicity_per_residue_stdev = []

for i in helicity_tuple:
    avg_helicity_per_residue.append(np.mean(i))
    avg_helicity_per_residue_stdev.append(np.std(i))


In [ ]:
fig,ax = plt.subplots()
plt.ylabel('%Helicity per Residue', fontsize=14)
plt.xticks(rotation=85, fontsize=12)
plt.yticks(fontsize=12)

ax.set_ylim([0.0, 105.0])
plt.gcf().set_size_inches(15,5)

ax.errorbar(peptide_res_renumbered[ace_cap:peptide_res_number], avg_helicity_per_residue, yerr=avg_helicity_per_residue_stdev/np.sqrt(3), fmt='none', ecolor='black', elinewidth=1, capsize=1)
ax.plot(peptide_res_renumbered[ace_cap:peptide_res_number], avg_helicity_per_residue, color='cadetblue', linewidth=2) 

fig.savefig(work_dir+'avg_helicity_per_residue_with_standarderror_Phe7_S51_S52_paperfigure.png', bbox_inches='tight')
plt.show()

#### RMSF calculations of the peptide (only for prod)

In [ ]:
heat1_traj_peptide = md.load_netcdf(work_dir+'heat1_dry.nc', top=work_dir+prmtop_name, atom_indices=peptide_atom_idx)
#first frame of heating 1 trajectory to be used for comparison
# keep the trajectory loaded to heat1.nc

In [ ]:
rmsf_c_alphas_peptide = md.rmsf(traj_peptide, heat1_traj_peptide, 0, atom_indices = [a.index for a in traj_peptide.topology.atoms if a.name == 'CA'])

In [ ]:
# convert to Angstroms

rmsf_c_alphas_peptide_A = []

for i in rmsf_c_alphas_peptide:
    rmsf_c_alphas_peptide_A.append(10*i)

In [ ]:
fig,ax = plt.subplots()
ax.plot(peptide_res_renumbered[ace_cap:peptide_res_number], rmsf_c_alphas_peptide_A)
ax.set(ylabel='RMSF ($\AA$)', title='RMSF of '+system_name)
plt.xticks(rotation=85, fontsize=5)

plt.gcf().set_size_inches(15,5)
#ax.set_ylim([0.0, 110.0])
fig.savefig(work_dir+'rmsf_peptide.png', bbox_inches='tight')
plt.show()

In [ ]:
# exclude initial (usually 2) residues to get a clearer picture about flexibility

fig,ax = plt.subplots()
ax.plot(peptide_res_renumbered[ace_cap+1:peptide_res_number], rmsf_c_alphas_peptide_A[1:])
ax.set(ylabel='RMSF ($\AA$)', title='RMSF of '+system_name)
plt.xticks(rotation=85, fontsize=5)

plt.gcf().set_size_inches(15,5)
ax.set_ylim([0.0, 10.0])
fig.savefig(work_dir+'rmsf_peptide_zoom.png', bbox_inches='tight')
plt.show()

#### RMSF calculation for the receptor

In [ ]:
traj_receptor1 = md.load_netcdf(work_dir+nc_file, top=work_dir+prmtop_name, atom_indices=receptor_atom_idx, stride = None)

In [ ]:
#traj_receptor2 = md.load_netcdf(work_dir+nc_file2, top=work_dir+prmtop_name, atom_indices=receptor_atom_idx, stride = 10)

In [ ]:
#traj_receptor3 = md.load_netcdf(work_dir+nc_file3, top=work_dir+prmtop_name, atom_indices=receptor_atom_idx, stride = 10)

In [ ]:
traj_receptor = traj_receptor1

In [ ]:
heat1_traj_receptor = md.load_netcdf(work_dir+'heat1_dry.nc', top=work_dir+prmtop_name, atom_indices=receptor_atom_idx)
#first frame of heating 1 trajectory to be used for comparison

In [ ]:
rmsf_c_alphas_receptor = md.rmsf(traj_receptor, heat1_traj_receptor, 0, atom_indices = [a.index for a in traj_receptor.topology.atoms if a.name == 'CA'])

In [ ]:
# convert to Angstroms

rmsf_c_alphas_receptor_A = []

for i in rmsf_c_alphas_receptor:
    rmsf_c_alphas_receptor_A.append(10*i)

In [ ]:
# make a list to exclude the NME indices from the receptor residue list

receptor_indices_without_nme = list(range(0, len(receptor_res)))

for i in receptor_indices_without_nme:
    if i in nme_res_list:
        receptor_indices_without_nme.remove(i)

print(receptor_indices_without_nme)



In [ ]:
fig,ax = plt.subplots()
ax.plot(receptor_indices_without_nme[1:], rmsf_c_alphas_receptor_A)
ax.set(xlabel = 'AMPA receptor residues')
ax.set(ylabel='RMSF ($\AA$)', title='RMSF of receptor in '+ system_name)
plt.xticks(rotation=85, fontsize=10)
plt.xticks(range(0,len(receptor_res),50))

plt.gcf().set_size_inches(15,5)
#ax.set_ylim([0.0, 110.0])
fig.savefig(work_dir+'rmsf_receptor.png', bbox_inches='tight')
plt.show()

In [ ]:
fig,ax = plt.subplots()
ax.plot(receptor_indices_without_nme[1:], rmsf_c_alphas_receptor_A)
ax.set(xlabel = 'AMPA receptor residues')
ax.set(ylabel='RMSF ($\AA$)', title='RMSF of receptor in '+ system_name)
plt.xticks(rotation=85, fontsize=10)
plt.xticks(range(0,len(receptor_res),50))

plt.gcf().set_size_inches(15,5)
ax.set_ylim([0.0, 30.0])
fig.savefig(work_dir+'rmsf_receptor_zoom.png', bbox_inches='tight')
plt.show()

#### RMSD calculations of the peptide

In [ ]:
t = np.arange(0.0, 200.0, 0.01) # length of simulation, change for different stages. Timestep is 2 fs.

In [ ]:
rmsd_backbone_peptide = md.rmsd(traj_peptide, heat1_traj_peptide, 0, atom_indices = [a.index for a in traj_peptide.topology.atoms if a.is_backbone])

In [ ]:
# convert to Angstrom

rmsd_backbone_peptide_A = []

for i in rmsd_backbone_peptide:
    rmsd_backbone_peptide_A.append(10*i)
    

In [ ]:
rmsd_backbone_peptide_A = rmsd_backbone_peptide_A[:len(t)]  # If error about prod trajectory being too long, use this line
fig, ax = plt.subplots()

# Customize the color of the plot points
color = 'royalblue'  # Change this to your desired color (e.g., 'red', 'green', '#FF5733', etc.)
ax.plot(t, rmsd_backbone_peptide_A, color=color)

fontsize = 16 
ax.set(xlabel='t (ns)', ylabel='RMSD ($\AA$)', title='RMSD of ' + system_name)
ax.xaxis.label.set_fontsize(fontsize)
ax.yaxis.label.set_fontsize(fontsize)
ax.title.set_fontsize(fontsize)

ax.set_ylim([0.0, 4.0])

ax.tick_params(axis='both', which='major', labelsize=fontsize)
fig.savefig(work_dir + 'rmsd_peptide.png', bbox_inches='tight')
plt.show()


#### RMSD calculation for the receptor

In [ ]:
rmsd_backbone_receptor = md.rmsd(traj_receptor, heat1_traj_receptor, 0, atom_indices = [a.index for a in traj_receptor.topology.atoms if a.is_backbone])

In [ ]:
rmsd_backbone_receptor

In [ ]:
# convert to Angstrom
rmsd_backbone_receptor_A = []
 
for i in rmsd_backbone_receptor:
    rmsd_backbone_receptor_A.append(10*i)

In [ ]:
rmsd_backbone_receptor_A = rmsd_backbone_receptor_A[:len(t)]     #if error about prod trajectory being too long, use this line
fig,ax = plt.subplots()

ax.plot(t, rmsd_backbone_receptor_A)

fontsize = 16 
ax.set(xlabel='t (ns)', ylabel='RMSD ($\AA$)', title='RMSD of receptor in '+system_name)
ax.xaxis.label.set_fontsize(fontsize)
ax.yaxis.label.set_fontsize(fontsize)
ax.title.set_fontsize(fontsize)

ax.set_ylim([0.0, 5.0])
ax.tick_params(axis='both', which='major', labelsize=fontsize)
fig.savefig(work_dir+'rmsd_receptor.png', bbox_inches='tight')
plt.show()

#### Hydrogen bond analysis

##### Hydrogen bonds in the peptide helix (8-18) (intrahelical hydrogen bonds)

In [ ]:
#helical part of CGRP peptide is Val8 to Arg18

helix_res = peptide_res[ace_cap:25] #[ace_cap:11] for 8-37, [7:18] for 1-37, [ace_cap:12] for Ac-8-37, [ace_cap:13] for Ac-Trp7-8-37
# to be automated
helix_res

In [ ]:
# renumbering the helix residues (8-18)

helix_res_renumbered = []


if peptide_res[0][0:3] == 'ACE' and peptide_res[1][0:3] == 'VAL':
    idx=7+1
elif peptide_res[0][0:3] == 'ACE' and peptide_res[1][0:3] == 'TRP':
    idx=6+1
elif peptide_res[0][0:3] == 'ACE' and peptide_res[1][0:3] == 'ALA':
    idx=6+1
elif peptide_res[0][0:3] == 'ACE' and peptide_res[1][0:3] == 'TYR':
    idx=6+1
elif peptide_res[0][0:3] == 'ACE' and peptide_res[1][0:3] == 'HIS':
    idx=6+1
elif peptide_res[0][0:3] == 'ACE' and peptide_res[1][0:3] == 'ORG':
    idx=7+1
elif peptide_res[0][0:3] == 'ACE' and peptide_res[1][0:3] == 'ORA':
    idx=7+1
elif peptide_res[0][0:3] == 'ACE' and peptide_res[1][0:3] == 'PHE':
    idx=6+1
elif peptide_res[0][0:3] == 'ACE' and peptide_res[1][0:3] == 'GLU':
    idx=1
elif peptide_res[0][0:3] == 'VAL':
    idx=8
elif peptide_res[0][0:3] == 'PHE':
    idx=6
elif peptide_res[0][0:3] == 'GLN':
    idx=6
elif peptide_res[0][0:3] == 'ALA':
    idx=1
elif peptide_res[0][0:3] == 'HIS':
    idx=1
    ace_cap = 0
elif peptide_res[0][0:3] == 'CYS':
    idx=1
    ace_cap = 0
elif peptide_res[0][0:3] == 'GLU':
    idx=1
    ace_cap = 0

for i in helix_res:
    helix_res_renumbered.append(i[0:3]+' '+str(idx))
    idx +=1

helix_res_renumbered


In [ ]:
helix_selection_mdtraj = 'resid '+str(helix_res[0][3:6])+' to '+str(helix_res[-1][3:6])
print(helix_selection_mdtraj)

In [ ]:
helix_atom_idx = traj.topology.select(helix_selection_mdtraj)

In [ ]:
# loading the helix trajectory

traj_helix1 = md.load_netcdf(work_dir+nc_file, top=work_dir+prmtop_name, atom_indices=helix_atom_idx)

In [ ]:
#traj_helix2 = md.load_netcdf(work_dir+nc_file2, top=work_dir+prmtop_name, atom_indices=helix_atom_idx)

In [ ]:
#traj_helix3 = md.load_netcdf(work_dir+nc_file3, top=work_dir+prmtop_name, atom_indices=helix_atom_idx)

In [ ]:
traj_helix = traj_helix1

Function for calculatung intrahelical hydrogen bonds for a given percentage of frames

In [ ]:
def intrahelical_hbonds(frame_percentage):
    hbonds_peptide = md.baker_hubbard(traj_helix, freq=frame_percentage/100, exclude_water=True, periodic=True, sidechain_only=False, distance_cutoff=0.25, angle_cutoff=120)

    label = lambda hbonds_peptide : '%s -- %s' % (traj_helix.topology.atom(hbond[0]), traj_helix.topology.atom(hbond[2]))

    hbonds_list = []
    for hbond in hbonds_peptide:
        hbonds_list.append(label(hbond))

    # removing hydrogen bonds where side chains are involved
    # keeping only backbone hydrogen bonds

    hbonds_list_backbone=[]

    for i in hbonds_list:
        if len(i) == 20: 
            hbonds_list_backbone.append(i)
    
    hbonds_pd_list = []  # hydrogen bonds list for pandas dataframe
    idx=0
    
    for i in hbonds_list_backbone:
        hbonds_pd_list.append([hbonds_list_backbone[idx][0:6], hbonds_list_backbone[idx][12:18], frame_percentage])
        idx+=1
    

    return hbonds_pd_list

In [ ]:
intrahelical_hbonds(1)

In [ ]:
#for i in intrahelical_hbonds(10):
    #df_test.loc[(df_test['N'] == i[0]) & (df_test['O'] == i[1]), '% frames'] = i[2]
    

In [ ]:
df=1 # clear 

In [ ]:
df = pd.DataFrame(intrahelical_hbonds(1), columns=['N', 'O', '% frames'])
df

In [ ]:
for j in range(2,101):
    for i in intrahelical_hbonds(j):
        df.loc[(df['N'] == i[0]) & (df['O'] == i[1]), '% frames'] = i[2]

In [ ]:
df

Making the intrahelical hydrogen bonds heatmap

In [ ]:
# creating all possible pairs of residues to add to the dataframe

from itertools import combinations

helix_res_tuples = list(combinations(helix_res, 2))


In [ ]:
helix_res_combs = []

for i in helix_res_tuples:
    lst = list(i)
    helix_res_combs.append(lst)

In [ ]:
helix_res_combs

In [ ]:
# add non-existent pairs to dataframe

# first remove the dataframe pairs from the helix_res_combs list
# finding which pairs exist in the dataframe

dataframe_pairs = []

for i in df.values:
    dataframe_pairs.append([i[0],i[1]])

dataframe_pairs
    

In [ ]:
# re-organising the dataframes pair list so upstream residues appear first in each pair

ordering = [1,0]

for lst in dataframe_pairs:
    lst[:] = [lst[i] for i in ordering]

In [ ]:
dataframe_pairs

In [ ]:
helix_res_combs_without_dataframe = copy.deepcopy(helix_res_combs)

for i in dataframe_pairs:
    if i in helix_res_combs_without_dataframe:
        helix_res_combs_without_dataframe.remove(i)

In [ ]:
helix_res_combs_without_dataframe

In [ ]:
# add 0% frame appearance for the rest of the hydrogen bond pairs
for i in helix_res_combs_without_dataframe:
    i.append(0)
helix_res_combs_without_dataframe

In [ ]:
# add remaining pairs to dataframe

for i in helix_res_combs_without_dataframe:
    df.loc[len(df)] = i

df

In [ ]:
# adding the inverted dataframe residues again, to have a symmetric heatmap

dataframe_inverted = []

for i in df.values:
    dataframe_inverted.append([i[0],i[1], i[2]])

new_ordering = [1,0,2]

for lst in dataframe_inverted:
    lst[:] = [lst[i] for i in new_ordering]

dataframe_inverted

In [ ]:
for i in dataframe_inverted:
    df.loc[len(df)] = i

df

In [ ]:
#data = df.pivot(index='N', columns='O', values='% frames')
data = df.pivot_table(index='N', columns='O', values='% frames', aggfunc='mean')  # or another aggfunc like sum, max


In [ ]:
data2 = copy.deepcopy(data)
data3 = data2.reindex(helix_res, axis=0)
data4 = data3.reindex(helix_res, axis=1)
data4

In [ ]:
x_axis_labels = helix_res_renumbered
y_axis_labels = helix_res_renumbered

In [ ]:
ax = sns.heatmap(data4, xticklabels=x_axis_labels, yticklabels=y_axis_labels, vmin=0, vmax=100, cmap='Reds', cbar_kws={'label': '% frames'})

# Set axis labels with font size
plt.xlabel('O-acceptor residue', fontsize=8)
plt.ylabel('N-donor residue', fontsize=8)
plt.title('Intrahelical hydrogen bond heatmap of ' + system_name, fontsize=15)

# Set the tick label font sizes
#ax.set_xticklabels(ax.get_xticklabels(), fontsize=7)
#ax.set_yticklabels(ax.get_yticklabels(), fontsize=7)

# Set figure size
plt.gcf().set_size_inches(10, 10)

# Save the figure
plt.savefig(work_dir+'intrahelical-hydrogen-bond-heatmap.png', bbox_inches='tight')
plt.show()


Creating average intrahelical h.bonds heatmap for all 3 replicates

Replicate 2

In [ ]:
traj_helix_rep2 = md.load_netcdf(work_dir+'Replicate-2/'+nc_file, top=work_dir+prmtop_name, atom_indices=helix_atom_idx)

In [ ]:
def intrahelical_hbonds_rep2(frame_percentage):
    hbonds_peptide = md.baker_hubbard(traj_helix_rep2, freq=frame_percentage/100, exclude_water=True, periodic=True, sidechain_only=False, distance_cutoff=0.25, angle_cutoff=120)

    label = lambda hbonds_peptide : '%s -- %s' % (traj_helix_rep2.topology.atom(hbond[0]), traj_helix_rep2.topology.atom(hbond[2]))

    hbonds_list = []
    for hbond in hbonds_peptide:
        hbonds_list.append(label(hbond))

    # removing hydrogen bonds where side chains are involved
    # keeping only backbone hydrogen bonds

    hbonds_list_backbone=[]

    for i in hbonds_list:
        if len(i) == 20: 
            hbonds_list_backbone.append(i)
    
    hbonds_pd_list = []  # hydrogen bonds list for pandas dataframe
    idx=0
    
    for i in hbonds_list_backbone:
        hbonds_pd_list.append([hbonds_list_backbone[idx][0:6], hbonds_list_backbone[idx][12:18], frame_percentage])
        idx+=1
    

    return hbonds_pd_list

In [ ]:
df_rep2 = 1 # for clearing

In [ ]:
df_rep2 = pd.DataFrame(intrahelical_hbonds_rep2(1), columns=['N', 'O', '% frames'])
df_rep2

In [ ]:
for j in range(2,101):
    for i in intrahelical_hbonds_rep2(j):
        df_rep2.loc[(df_rep2['N'] == i[0]) & (df_rep2['O'] == i[1]), '% frames'] = i[2]

In [ ]:
df_rep2

In [ ]:
# add non-existent pairs to dataframe

# first remove the dataframe pairs from the helix_res_combs list
# finding which pairs exist in the dataframe

dataframe_pairs_rep2 = []

for i in df_rep2.values:
    dataframe_pairs_rep2.append([i[0],i[1]])

dataframe_pairs_rep2
    

In [ ]:
# re-organising the dataframes pair list so upstream residues appear first in each pair

ordering = [1,0]

for lst in dataframe_pairs_rep2:
    lst[:] = [lst[i] for i in ordering]

In [ ]:
dataframe_pairs_rep2

In [ ]:
helix_res_combs_without_dataframe_rep2 = copy.deepcopy(helix_res_combs)

for i in dataframe_pairs_rep2:
    if i in helix_res_combs_without_dataframe_rep2:
        helix_res_combs_without_dataframe_rep2.remove(i)

In [ ]:
helix_res_combs_without_dataframe_rep2

In [ ]:
# add 0% frame appearance for the rest of the hydrogen bond pairs
for i in helix_res_combs_without_dataframe_rep2:
    i.append(0)
helix_res_combs_without_dataframe_rep2

In [ ]:
# add remaining pairs to dataframe

for i in helix_res_combs_without_dataframe_rep2:
    df_rep2.loc[len(df_rep2)] = i

df_rep2

In [ ]:
# adding the inverted dataframe residues again, to have a symmetric heatmap

dataframe_inverted_rep2 = []

for i in df_rep2.values:
    dataframe_inverted_rep2.append([i[0],i[1], i[2]])

new_ordering = [1,0,2]

for lst in dataframe_inverted_rep2:
    lst[:] = [lst[i] for i in new_ordering]

dataframe_inverted_rep2

In [ ]:
for i in dataframe_inverted_rep2:
    df_rep2.loc[len(df_rep2)] = i

df_rep2

In [ ]:
data_rep2 = df_rep2.pivot(index='N', columns='O', values='% frames')

In [ ]:
data2_rep2 = copy.deepcopy(data_rep2)
data3_rep2 = data2_rep2.reindex(helix_res, axis=0)
data4_rep2 = data3_rep2.reindex(helix_res, axis=1)
data4_rep2

Replicate 3

In [ ]:
traj_helix_rep3 = md.load_netcdf(work_dir+'Replicate-3/'+nc_file, top=work_dir+prmtop_name, atom_indices=helix_atom_idx)

In [ ]:
def intrahelical_hbonds_rep3(frame_percentage):
    hbonds_peptide = md.baker_hubbard(traj_helix_rep3, freq=frame_percentage/100, exclude_water=True, periodic=True, sidechain_only=False, distance_cutoff=0.25, angle_cutoff=120)

    label = lambda hbonds_peptide : '%s -- %s' % (traj_helix_rep3.topology.atom(hbond[0]), traj_helix_rep3.topology.atom(hbond[2]))

    hbonds_list = []
    for hbond in hbonds_peptide:
        hbonds_list.append(label(hbond))

    # removing hydrogen bonds where side chains are involved
    # keeping only backbone hydrogen bonds

    hbonds_list_backbone=[]

    for i in hbonds_list:
        if len(i) == 20: 
            hbonds_list_backbone.append(i)
    
    hbonds_pd_list = []  # hydrogen bonds list for pandas dataframe
    idx=0
    
    for i in hbonds_list_backbone:
        hbonds_pd_list.append([hbonds_list_backbone[idx][0:6], hbonds_list_backbone[idx][12:18], frame_percentage])
        idx+=1
    

    return hbonds_pd_list

In [ ]:
df_rep3 = 1 # for clearing

In [ ]:
df_rep3 = pd.DataFrame(intrahelical_hbonds_rep3(1), columns=['N', 'O', '% frames'])
df_rep3

In [ ]:
for j in range(2,101):
    for i in intrahelical_hbonds_rep3(j):
        df_rep3.loc[(df_rep3['N'] == i[0]) & (df_rep3['O'] == i[1]), '% frames'] = i[2]

In [ ]:
df_rep3

In [ ]:
# add non-existent pairs to dataframe

# first remove the dataframe pairs from the helix_res_combs list
# finding which pairs exist in the dataframe

dataframe_pairs_rep3 = []

for i in df_rep3.values:
    dataframe_pairs_rep3.append([i[0],i[1]])

dataframe_pairs_rep3
    

In [ ]:
# re-organising the dataframes pair list so upstream residues appear first in each pair

ordering = [1,0]

for lst in dataframe_pairs_rep3:
    lst[:] = [lst[i] for i in ordering]

In [ ]:
dataframe_pairs_rep3

In [ ]:
helix_res_combs_without_dataframe_rep3 = copy.deepcopy(helix_res_combs)

for i in dataframe_pairs_rep3:
    if i in helix_res_combs_without_dataframe_rep3:
        helix_res_combs_without_dataframe_rep3.remove(i)

In [ ]:
# add 0% frame appearance for the rest of the hydrogen bond pairs
for i in helix_res_combs_without_dataframe_rep3:
    i.append(0)
helix_res_combs_without_dataframe_rep3

In [ ]:
# add remaining pairs to dataframe

for i in helix_res_combs_without_dataframe_rep3:
    df_rep3.loc[len(df_rep3)] = i

df_rep3

In [ ]:
# adding the inverted dataframe residues again, to have a symmetric heatmap

dataframe_inverted_rep3 = []

for i in df_rep3.values:
    dataframe_inverted_rep3.append([i[0],i[1], i[2]])

new_ordering = [1,0,2]

for lst in dataframe_inverted_rep3:
    lst[:] = [lst[i] for i in new_ordering]

dataframe_inverted_rep3

In [ ]:
for i in dataframe_inverted_rep3:
    df_rep3.loc[len(df_rep3)] = i

df_rep3

In [ ]:
data_rep3 = df_rep3.pivot(index='N', columns='O', values='% frames')

In [ ]:
data2_rep3 = copy.deepcopy(data_rep3)
data3_rep3 = data2_rep3.reindex(helix_res, axis=0)
data4_rep3 = data3_rep3.reindex(helix_res, axis=1)
data4_rep3

Average intrahelical h.bonds heatmap from the 3 replicates

In [ ]:
data4.to_csv(work_dir+'rep1.csv')
data4_rep2.to_csv(work_dir+'rep2.csv')
data4_rep3.to_csv(work_dir+'rep3.csv')

In [ ]:
# offline processing in Excel

In [ ]:
avg_df = pd.read_csv(work_dir+'all_reps.csv')

In [ ]:
avg_df

In [ ]:
ax = sns.heatmap(avg_df, xticklabels=x_axis_labels, yticklabels=y_axis_labels, vmin=0, vmax=100, cmap = 'Reds', cbar_kws= {'label': '% frames'})

plt.xlabel('O-acceptor residue', fontsize = 10)
plt.ylabel('N-donor residue', fontsize = 10)
plt.title('Intrahelical hydrogen bond heatmap of '+system_name)

plt.gcf().set_size_inches(6,6)

plt.savefig(work_dir+'avg_intrahelical-hydrogen-bond-heatmap.png', bbox_inches='tight')
plt.show()

##### Hydrogen bonds in peptide-receptor system

In [ ]:
hbonds_system = md.baker_hubbard(traj, freq=0.1, exclude_water=True, periodic=True, sidechain_only=True, distance_cutoff=0.25, angle_cutoff=120)

In [ ]:
label = lambda hbonds_system : '%s -- %s' % (traj.topology.atom(hbond[0]), traj.topology.atom(hbond[2]))

for hbond in hbonds_system:
    print(label(hbond))

#### Finding the active site

In [ ]:
u = mda.Universe(work_dir+prmtop_name, work_dir+'heat1.nc')

In [ ]:
# automate residue selection

peptide_mda = u.select_atoms('resid 457:487') 
# residue numbering in MDAnalysis starts from 1, so the residue selection should be +1 compared to the mdtraj selection

In [ ]:
print(peptide_mda.residues)

In [ ]:
active_site = u.select_atoms('protein and around 5.0 resid 457:487')

In [ ]:
for res in active_site.residues:
    print(res)

In [ ]:
# these residues will be useful for PyContact analysis

### Representative Structure for Full Protein (Clustering, production only)

In [ ]:
protein_n_ter = int(str(protein_res_strings[0])[3:7])
protein_c_ter = int(str(protein_res_strings[-1])[3:7])
print(protein_n_ter, protein_c_ter)

In [ ]:
protein_selection_mdtraj = 'resid '+str(protein_n_ter)+' to '+str(protein_c_ter)
print('mdtraj selection for the peptide: '+protein_selection_mdtraj)

In [ ]:
protein_selection_mdanalysis = 'resid '+str(protein_n_ter+1)+':'+str(protein_c_ter+2)
print('MDAnalysis selection for the peptide: '+protein_selection_mdanalysis)

In [ ]:
protein_atom_idx = traj.topology.select(protein_selection_mdtraj)

In [ ]:
protein_traj1 = md.load_netcdf(work_dir+nc_file, top=work_dir+prmtop_name, atom_indices=protein_atom_idx, stride = 10)

In [ ]:
protein_traj2 = md.load_netcdf(work_dir+"Replicate-2/prod_dry.nc", top=work_dir+prmtop_name, atom_indices=protein_atom_idx, stride = 10)

In [ ]:
protein_traj3 = md.load_netcdf(work_dir+"Replicate-3/prod_dry.nc", top=work_dir+prmtop_name, atom_indices=protein_atom_idx, stride = 10)

In [ ]:
#protein_traj4 = md.load_netcdf(work_dir+nc_file4, top=work_dir+prmtop_name, atom_indices=protein_atom_idx, stride = 10)

In [ ]:
protein_traj = protein_traj1+protein_traj2+protein_traj3

In [ ]:
protein_traj

In [ ]:
atom_indices = [a.index for a in protein_traj.topology.atoms if a.element.symbol != 'H']
distances = np.empty((protein_traj.n_frames, protein_traj.n_frames))
for i in range(protein_traj.n_frames):
    print(i)
    distances[i] = md.rmsd(protein_traj, protein_traj, i, atom_indices=atom_indices)

In [ ]:
beta = 1
index = np.exp(-beta*distances / distances.std()).sum(axis=1).argmax()
print(index)

In [ ]:
centroid = protein_traj[index]
print(centroid)

In [ ]:
# view = nv.show_pytraj(centroid)
# view.add_cartoon()
# view

In [ ]:
centroid.save_pdb(work_dir+'Con-ikot-ikot_in_AMPAR_representative_structure.pdb')

### Representative Structure for Peptide (Clustering, production only)

In [ ]:
atom_indices = [a.index for a in traj_peptide.topology.atoms if a.element.symbol != 'H']
distances = np.empty((traj_peptide.n_frames, traj_peptide.n_frames))
for i in range(traj_peptide.n_frames):
    print(i)
    distances[i] = md.rmsd(traj_peptide, traj_peptide, i, atom_indices=atom_indices)

In [ ]:
beta = 1
index_peptide = np.exp(-beta*distances / distances.std()).sum(axis=1).argmax()
print(index_peptide)

In [ ]:
centroid_peptide = traj_peptide[index_peptide]
print(centroid_peptide)

In [ ]:
view = nv.show_pytraj(centroid_peptide)
view.add_cartoon()
view

In [ ]:
centroid_peptide.save_pdb(work_dir+'R30K_Ac-PACAP(6-38)_S5A24_S5G28_only_representative_structure.pdb')

### Calculate distance between two atoms over trajectory

In [ ]:
# Define atom indices for the distance calculation (0-based indexing)
# Replace 0 and 1 with the indices of the two atoms of interest
atom1_index = 3177  # First atom index
atom2_index = 7140  # Second atom index
distance_threshold = 4

# Calculate the distance between the two atoms across all frames
distances = md.compute_distances(traj, [[atom1_index, atom2_index]])

# Convert distances from nanometres to angstroms
distances_angstrom = distances * 10

# Average distance and standard deviation
average_distance = np.mean(distances_angstrom)
std_distance = np.std(distances_angstrom)

# Calculate percentage of frames with distance below 4 angstroms
below_4_angstrom = np.sum(distances_angstrom < distance_threshold)
total_frames = len(distances_angstrom)
percentage_below_4 = (below_4_angstrom / total_frames) * 100

# Print results
print("Distances between atoms {} and {} (in angstroms):".format(atom1_index, atom2_index))
print(distances_angstrom)
print("\nAverage distance: {:.3f} Å".format(average_distance))
print("Standard deviation: {:.3f} Å".format(std_distance))
print("Percentage of frames with distance below 4 Å: {:.2f}%".format(percentage_below_4))

# Save distances to a file
file_name = "distances_between_atom{}_atom{}_over_traj.txt".format(atom1_index, atom2_index)
with open(work_dir+file_name, "w") as f:
    f.write("Distances (Å) between atoms {} and {}:\n".format(atom1_index, atom2_index))
    f.write("\nAverage distance: {:.3f} Å\n".format(average_distance))
    f.write("Standard deviation: {:.3f} Å\n\n".format(std_distance))
    f.write("Percentage of frames with distance below 4 Å: {:.2f}%\n\n".format(percentage_below_4))
    np.savetxt(f, distances_angstrom)
    

